In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F

In [2]:
hotel_1 = pd.read_csv('H1.csv',parse_dates=True,index_col='ReservationStatusDate')
hotel_2 = pd.read_csv('H2.csv',parse_dates=True,index_col='ReservationStatusDate')

In [3]:
hotel_1 = hotel_1.replace(to_replace = '       NULL', 
                 value =np.NAN) 
hotel_2 = hotel_2.replace(to_replace = '       NULL', 
                 value =np.NAN) 

In [4]:
# Drop Company and Agent from both hotel_1 & hotel_2 datasets
hotel_1 = hotel_1.drop(['Agent','Company'],axis=1)
hotel_2 = hotel_2.drop(['Agent','Company'],axis=1)

# Fill NA values using Most frequently occuring value in that column
hotel_1['Country'] = hotel_1['Country'].fillna(hotel_1['Country'].mode()[0])

hotel_2['Country'] = hotel_2['Country'].fillna(hotel_2['Country'].mode()[0])
hotel_2['Children'] = hotel_2['Children'].fillna(hotel_2['Children'].mode()[0])

In [5]:
# drop arrival date month
hotel_1 = hotel_1.drop(['ArrivalDateMonth'],axis=1)
hotel_2 = hotel_2.drop(['ArrivalDateMonth'],axis=1)

# drop arrival date day of month
hotel_1 = hotel_1.drop(['ArrivalDateDayOfMonth'],axis=1)
hotel_2 = hotel_2.drop(['ArrivalDateDayOfMonth'],axis=1)

# drop reservation status
hotel_1 = hotel_1.drop(['ReservationStatus'],axis=1)
hotel_2 = hotel_2.drop(['ReservationStatus'],axis=1)

In [6]:
hotel_1['AssignNewRoom'] = 0
# check if the reserved room type is different from assigned room type
hotel_1.loc[hotel_1['ReservedRoomType'] != hotel_1['AssignedRoomType'], 'AssignNewRoom'] = 1
# drop older features
hotel_1 = hotel_1.drop(['AssignedRoomType', 'ReservedRoomType'], axis=1)

In [7]:
# replacing 1 by True and 0 by False for treatment and outcome features
hotel_1['AssignNewRoom'] = hotel_1['AssignNewRoom'].replace({1: True, 0: False})
hotel_1['IsCanceled'] = hotel_1['IsCanceled'].replace({1: True, 0: False})

In [8]:
t = torch.tensor(hotel_1['AssignNewRoom'].values, dtype=torch.float32).reshape(-1,1)
y = torch.tensor(hotel_1['IsCanceled'].values, dtype=torch.float32).reshape(-1,1)
concat_true = torch.cat((y,t),1)
w = torch.tensor(hotel_1['BookingChanges'].values, dtype=torch.float32).reshape(-1,1)

In [9]:
g = torch.Generator().manual_seed(42)
np.random.seed(42)

In [10]:
10 % 3

1

In [11]:
from models import tarnet

In [12]:
ate = tarnet.estimate(
    treatment = t,
    outcome = y,
    confounders = w,
)
ate

Iteration 0 loss: 12.239068031311035
Iteration 100 loss: 5.073991298675537
Iteration 200 loss: 6.283305644989014
Iteration 300 loss: 5.797115325927734
Iteration 400 loss: 5.619232177734375
Iteration 500 loss: 4.334939002990723
Iteration 600 loss: 6.6093339920043945
Iteration 700 loss: 6.332919120788574
Iteration 800 loss: 5.219549655914307
Iteration 900 loss: 4.644416809082031
Iteration 1000 loss: 6.156187057495117
Iteration 1100 loss: 5.482533931732178
Iteration 1200 loss: 6.653475284576416
Iteration 1300 loss: 6.527281284332275
Iteration 1400 loss: 6.960143089294434
Iteration 1500 loss: 5.749301433563232
Iteration 1600 loss: 4.6785197257995605
Iteration 1700 loss: 7.3059234619140625
Iteration 1800 loss: 6.570338249206543
Iteration 1900 loss: 6.236927509307861
Iteration 2000 loss: 6.022600173950195


-0.23259851336479187

In [13]:
# -0.2538311183452606
# 12.284306526184082